# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset_chokepoint' #'colab_main_dataset'
input_dir = os.path.join(home,'image_datasets')
output_dir = os.path.join(home,'sql_database')
#input_dir = os.path.join(home,'Desktop', 'image_datasets')
#output_dir = os.path.join(home,'Desktop','sql_database')
database_names=[ 'chokepoint'] # 'lfw', 'scface', 'forenface', 'enfsi', 'enfsi2015' 'xqlfw', 'utkface','chokepoint'
detector_names=['mtcnn', 'mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab



## SERFIQ model

Create SERFIQ model.

In [ ]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [ ]:
from sql_face.sqldb import SQLDataBase

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [ ]:
database.__dict__

In [ ]:
database.create_tables(serfiq=serfiq)

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)